In [3]:
import glob
import pickle
import gzip

used for 11093a30-b6d0-4e3f-a22b-8dcad60d6a11_100

In [8]:
molregnos,mols,errors = [],[],[]
for file in glob.glob('./confgen/*.pkl.gz'):
    with gzip.open(file,'rb') as inf:
        ind =pickle.load(inf)
        for mol in ind:
            molregnos.append(mol[0])
            mols.append(mol[1])
            errors.append(mol[2])
print(len(molregnos),len(mols),len(errors))

3019 3019 3019


In [34]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.SimDivFilters import rdSimDivPickers

picker = rdSimDivPickers.MaxMinPicker()

def pick_conformers(mol,picks):
    def custom_rmsd_distance(i, j):
        return AllChem.GetBestRMS(mol,mol,i,j)
    selected_indices = list(picker.LazyPick(custom_rmsd_distance, mol.GetNumConformers(), picks, seed=1966))
    mol_X_confs = Chem.Mol(mol)
    mol_X_confs.RemoveAllConformers()
    for i in selected_indices:
        mol_X_confs.AddConformer(mol.GetConformer(i))
    return mol_X_confs

In [44]:
#okay lets start the first five molecules
from tqdm import tqdm
selected_confs_mol_list = []
outfn = './11093a30-b6d0-4e3f-a22b-8dcad60d6a11_5_100.pkl.gz'
for mol,molregno in tqdm(zip(mols,molregnos)):
    mol = Chem.RemoveHs(mol)    
    if mol.GetNumConformers() > 5:
        picked_mol = pick_conformers(mol,5)
        selected_confs_mol_list.append(picked_mol)
    else:
        selected_confs_mol_list.append(mol)
with gzip.open(outfn,'wb+') as outf:
    pickle.dump(selected_confs_mol_list,outf)

3019it [34:47,  1.45it/s] 


In [45]:
outfn = './11093a30-b6d0-4e3f-a22b-8dcad60d6a11_mrn_5_100.pkl.gz'
selected_confs_mol_mrn_tupple_list = []
for mol,molregno in zip(selected_confs_mol_list,molregnos):
    selected_confs_mol_mrn_tupple_list.append((mol,molregno))
with gzip.open(outfn,'wb+') as outf:
    pickle.dump(selected_confs_mol_mrn_tupple_list,outf)
    

In [49]:
#do some check; load the file
with gzip.open('./11093a30-b6d0-4e3f-a22b-8dcad60d6a11_mrn_5_100.pkl.gz','rb') as inf:
    loaded_list = pickle.load(inf)
    mols = [mol[0] for mol in loaded_list]
for mol in mols[:5]:
    print(mol.GetNumConformers())
    print([conf.GetId() for conf in mol.GetConformers()])
    


3
[0, 1, 2]
5
[14, 5, 8, 1, 2]
2
[0, 1]
5
[57, 56, 24, 5, 68]
5
[18, 17, 29, 10, 28]


In [51]:
#now we need to save these molecules in a sdf file
from tqdm import tqdm
with Chem.SDWriter('11093a30-b6d0-4e3f-a22b-8dcad60d6a11_5_100.sdf') as w:
    for mol in tqdm(mols):
        cids = [conf.GetId() for conf in mol.GetConformers()]
        for cid in cids:
            w.write(mol,confId=cid)


100%|██████████| 3019/3019 [00:00<00:00, 3840.46it/s]


In [52]:
mols_five_conf_sup = Chem.SDMolSupplier('11093a30-b6d0-4e3f-a22b-8dcad60d6a11_5_100.sdf',removeHs=False)
mols_five_conf = [mol for mol in mols_five_conf_sup]
print(len(mols_five_conf))

11389


In [47]:
#lets see what happens if we query for this mol
import lwreg
from lwreg import standardization_lib
from lwreg import utils
import psycopg2
import pandas as pd
import psycopg2
hostname = 'scotland'
dbname = 'cs_mdfps'
username = 'cschiebroek'
config = lwreg.utils.defaultConfig()
# set the name of the database we'll work with:
config['dbtype'] = 'postgresql'
config['dbname'] = 'cs_mdfps'
config['host'] = 'scotland'
config['user'] = 'cschiebroek'
config['password'] = '' # password is saved in our .pgpass
# we don't want to standardize the molecules:
config['standardization'] = standardization_lib.RemoveHs()
# we want to store conformers
config['registerConformers'] = True

In [53]:
#now we can register them
tupp_list =[]
alread_registered = []
for mol in tqdm(mols_five_conf_sup):
    try:
        tupp =lwreg.register(config=config, mol=mol,)
        tupp_list.append(tupp)
    except:
        alread_registered.append(mol)
print(len(tupp_list),len(alread_registered))


100%|██████████| 11389/11389 [09:55<00:00, 19.13it/s]

11388 1


In [54]:
#then register the conf gen metadata
import json
import psycopg2
cn = psycopg2.connect(host='scotland',dbname='cs_mdfps',user='cschiebroek')
cur = cn.cursor()
confgen_uuid = '11093a30-b6d0-4e3f-a22b-8dcad60d6a11'
params = {"randomSeed":0xa700f,"verbose":False,"numThreads":4,"trackFailures":True,"useRandomCoords":True,"pruneRmsThresh":0.1,"numConfs":100}
params_json = json.dumps(params)
cur.execute('insert into cs_mdfps_schema.confid_metadata values (%s,%s)',(str(confgen_uuid),str(params_json)))
cn.commit()



In [55]:
#and the seperate confs
new_confids = [t[1] for t in tupp_list]
#now register all confids in confid_data with confgen_uuid
for confid in new_confids:
    cur.execute('insert into cs_mdfps_schema.confid_data values (%s,%s)',(str(confid),str(confgen_uuid)))
cn.commit()


In [60]:
#then we need to make the files
#Get the molblocks for these new confids
cn = psycopg2.connect(host='scotland',dbname='cs_mdfps',user='cschiebroek')
cur = cn.cursor()
cur.execute('select * from conformers where conf_id in %s',(tuple(new_confids),))
result = cur.fetchall()
print(len(result))
print(result[0])
confdis = [r[0] for r in result]
molblocks = [r[3] for r in result]
for mb,confid in zip(molblocks,confdis):
    print(mb,file=open(f'mol_11093a30-b6d0-4e3f-a22b-8dcad60d6a11/{confid}.mol','w+'))

11388
(5781, 3096, '-0.272,-0.689,1.068;-0.497,1.73,1.272;-0.535,-1.855,1.76;-1.196,0.482,1.08;-1.842,1.374,-1.162;-2.047,0.483,-0.138;-2.971,-0.368,-0.201;0.34,-2.926,1.732;0.9,-0.633,0.347;1.514,-2.865,1.007;1.78,-1.711,0.319', '\n     RDKit          3D\n\n  0  0  0  0  0  0  0  0  0  0999 V3000\nM  V30 BEGIN CTAB\nM  V30 COUNTS 11 11 0 0 0\nM  V30 BEGIN ATOM\nM  V30 1 N -0.497300 1.730100 1.272300 0 CHG=1 VAL=4\nM  V30 2 C -1.195800 0.481600 1.080100 0 CFG=1\nM  V30 3 C -2.047400 0.482700 -0.138000 0\nM  V30 4 O -2.971100 -0.367700 -0.200900 0\nM  V30 5 O -1.842500 1.374100 -1.162400 0 CHG=-1 VAL=1\nM  V30 6 C -0.271600 -0.688900 1.068500 0\nM  V30 7 C -0.535000 -1.855200 1.760100 0\nM  V30 8 C 0.340500 -2.926400 1.731900 0\nM  V30 9 C 1.513500 -2.865400 1.007300 0\nM  V30 10 C 1.779900 -1.711100 0.319400 0\nM  V30 11 C 0.900400 -0.633100 0.346800 0\nM  V30 END ATOM\nM  V30 BEGIN BOND\nM  V30 1 1 2 1 CFG=3\nM  V30 2 1 2 3\nM  V30 3 2 3 4\nM  V30 4 1 3 5\nM  V30 5 1 2 6\nM  V30 6 2 6

In [ ]:
#set up the approriate data in databse for the md experiment; 
#actually, everything remains the same? but we could still make a new md_experiment uuid?

In [ ]:
#finally, we can start the simulations